In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Load the train-data CSV file
train_data = pd.read_csv('train-data.csv')

In [3]:
# Load the test-data CSV file
test_data = pd.read_csv('test-data.csv')

In [4]:
# Remove unnecessary columns (such as 'Unnamed: 0' and 'New_Price')
train_data = train_data.drop(['Unnamed: 0', 'New_Price'], axis=1)
test_data = test_data.drop(['Unnamed: 0', 'New_Price'], axis=1)

In [5]:
# Add a flag column to differentiate train and test datasets
train_data['Flag'] = 1
test_data['Flag'] = 0

In [6]:
# Combine train and test data for preprocessing
combined_data = pd.concat([train_data, test_data])

In [7]:
# Preprocess the data
combined_data['Mileage'] = combined_data['Mileage'].str.extract('(\d+\.\d+|\d+)').astype(float)
combined_data['Engine'] = combined_data['Engine'].str.extract('(\d+)').astype(float).fillna(0).astype(int)
combined_data['Power'] = combined_data['Power'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [8]:
# Convert categorical columns to numerical using LabelEncoder
label_encoder = LabelEncoder()
categorical_cols = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']
for col in categorical_cols:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

In [9]:
# Perform one-hot encoding on the combined dataset
onehot_encoder = OneHotEncoder(sparse=False)
encoded_data = onehot_encoder.fit_transform(combined_data)

In [10]:
# Split the combined data back into train and test datasets
X_train = encoded_data[combined_data['Flag'] == 1, :-1]
y_train = encoded_data[combined_data['Flag'] == 1, -1]
X_test = encoded_data[combined_data['Flag'] == 0, :-1]

In [11]:
# Fill missing values with 0
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)

In [12]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Build the ANN model with ReLU activation
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

In [20]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

In [21]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

Epoch 1/10
189/189 [==============================] - 2s 6ms/step - loss: 0.0169 - accuracy: 0.9940
Epoch 2/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0160 - accuracy: 0.9934
Epoch 3/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0174 - accuracy: 0.9935
Epoch 4/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0266 - accuracy: 0.9870
Epoch 5/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0150 - accuracy: 0.9953
Epoch 6/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0108 - accuracy: 0.9968
Epoch 7/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0171 - accuracy: 0.9915
Epoch 8/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0106 - accuracy: 0.9968
Epoch 9/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0055 - accuracy: 0.9982
Epoch 10/10
189/189 [==============================] - 1s 6ms/step - loss: 0.0042 - accuracy: 0.9985

In [22]:
# Predict on the test data
predictions = model.predict(X_test_scaled)

In [23]:
# Print the predicted prices
print(predictions)

[[0.9823433]
 [1.0029156]
 [1.0103791]
 ...
 [1.001    ]
 [0.9850092]
 [0.9820406]]
